#### **Imports and variables**

In [3]:
import os
import time
from representative_subset import greedy_representative_subset, greedy_representative_subset_v2, global_alignment_score, score
from data_io import parse_fasta, write_fasta
from alignment_viewer import view_alignment
import random
from Bio import AlignIO, SeqIO
from tqdm import tqdm
import panel as pn
import panel.widgets as pnw
pn.extension()

In [2]:
datasets_folder = './data'
aligned_datasets = {
    'H1N1': 'H1N1_Aligned.fasta',
    'H3N2': 'H3N2_Aligned.fasta',
    'H5N1': 'H5N1_Aligned.fasta',
    'H9N2': 'H9N2_Aligned.fasta',
}
unaligned_datasets = {
    'H1N1': 'H1N1_Seg4.fasta',
    'H3N2': 'H3N2_Seg4.fasta',
}

#### **Compare Algorithm 1 vs Algorithm 2 with aligned sequences**

In [3]:
for dataset in aligned_datasets:
    print(f"Dataset {dataset}:")
    input_path = os.path.join(datasets_folder, aligned_datasets[dataset])    
    sequences = parse_fasta(input_path)
    print("\tExtracting representative subset using Algorithm 1...")
    init = time.time()
    subset = greedy_representative_subset(sequences)
    end = time.time()
    print(f"\tExtracted {len(subset)} representative sequences from {len(sequences)} in {end-init} seconds.")
    print("\tExtracting representative subset using Algorithm 2...")
    init = time.time()
    subset_v2 = greedy_representative_subset_v2(sequences)
    end = time.time()
    print(f"\tExtracted {len(subset_v2)} representative sequences from {len(sequences)} input sequences in {end-init} seconds.")

Dataset H1N1:
	Extracting representative subset using Algorithm 1...


100%|█████████████████████████████████████| 2028/2028 [00:01<00:00, 1264.73it/s]


	Extracted 10 representative sequences from 2028 in 1.6220788955688477 seconds.
	Extracting representative subset using Algorithm 2...


100%|███████████████████████████████████████| 2028/2028 [09:53<00:00,  3.42it/s]


	Extracted 10 representative sequences from 2028 input sequences in 597.4331140518188 seconds.
Dataset H3N2:
	Extracting representative subset using Algorithm 1...


...

	Extracted 50 representative sequences from 1793 input sequences in 483.2029902935028 seconds.
Dataset H9N2:
	Extracting representative subset using Algorithm 1...


100%|██████████████████████████████████████| 2924/2924 [00:09<00:00, 310.61it/s]


	Extracted 166 representative sequences from 2924 in 9.918664932250977 seconds.
	Extracting representative subset using Algorithm 2...


100%|███████████████████████████████████████| 2924/2924 [20:31<00:00,  2.37it/s]


	Extracted 135 representative sequences from 2924 input sequences in 1336.290301322937 seconds.


# **Compare Algorithm 1 with/without aligned inputs**

In [4]:
for dataset in unaligned_datasets:
    print(f"Dataset {dataset}:")
    input_path = os.path.join(datasets_folder, unaligned_datasets[dataset])    
    sequences = parse_fasta(input_path)
    print("\tExtracting representative subset using Algorithm 1 without aligned sequences...")
    init = time.time()
    subset_v2 = greedy_representative_subset(sequences, aligned=False)
    end = time.time()
    print(f"\tExtracted {len(subset)} representative sequences from {len(sequences)} input sequences in {end-init} seconds.")

Dataset H1N1:
	Extracting representative subset using Algorithm 1 without aligned sequences...


100%|█████████████████████████████████████| 2028/2028 [2:44:40<00:00,  4.87s/it]


	Extracted 166 representative sequences from 2028 input sequences in 9882.432312011719 seconds.
Dataset H3N2:
	Extracting representative subset using Algorithm 1 without aligned sequences...


100%|█████████████████████████████████████| 1431/1431 [1:06:23<00:00,  2.78s/it]

	Extracted 166 representative sequences from 1431 input sequences in 3983.5353651046753 seconds.


### Benchmarking

In [ ]:
subset_list = []
test_list = []
for dataset in aligned_datasets:
    print(f"Dataset {dataset}:")
    input_path = os.path.join(datasets_folder, aligned_datasets[dataset])
    output_path = os.path.join(datasets_folder, aligned_datasets[dataset].replace(".fasta", "_Subset.fasta"))
    sequences = parse_fasta(input_path)
    print("\tSplitting training and testing sets...")
    testing_indices = random.choices(list(range(len(sequences))), k=len(sequences)//10)
    training_set = [sequences[i] for i in range(len(sequences)) if i not in testing_indices]
    testing_set = [sequences[i] for i in testing_indices]
    test_list.append(testing_set)
    write_fasta(testing_set, os.path.join(datasets_folder, dataset+'_testing.fasta'))
    print(f"\tSplit testing sequences to {dataset}_testing.fasta")
    print("\tExtracting representative subset using Algorithm 1...")
    init = time.time()
    subset = greedy_representative_subset(training_set)
    subset_list.append(subset)
    end = time.time()
    print(f"\tExtracted {len(subset)} representative sequences from {len(sequences)} input sequences in {end-init} seconds.")
    print(f"Writing results into {output_path}")
    write_fasta(subset, output_path)
    print("Done! \n")

accuracy_score = 0
test_size = 50
size = sum([test_size for i in test_list])
for i in range(len(test_list)):
    test = random.sample(test_list[i], 50)
    d_temp = len(test[0])*75//100
    subset = subset_list[i]
    for j in tqdm(range(test_size)):
        for k in subset:
            temp = global_alignment_score(test[j].replace('-', ''),k.replace('-', ''))
            if(temp>=d_temp):
                accuracy_score += 1
                break
print(f"Subset Accuracy: {accuracy_score/size}")